# Analisi della lunghezza dei suburn e sui motivi di stop

In [1]:
!jt -t grade3 -fs 90 -tfs 14 -nfs 115 -cellw 80% -T
import ROOT as R
import array
import numpy as np
%jsroot on
R.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.16/00


Ho creato un piccolo script che importa i log file di ogni subrun e li struttura in un tree fatto così:

                int runNo;\ 
                int subRunNo;\
                int triggers;\--> Totale trigger acquisiti dalla GEMROC che ne ha acquisiti meno (invece che la media o la moda, è cmq praticamente uguale)
                int error_end_FEB;\--> -1 se finito in altro modo, = a numero FEB che ha interrotto se finito per errori 8b/10b
                int standard_end;\--> 1 se finito normalmente
                int timeout_end;\--> 1 se finito per timeout
                int DAQ_crash;\--> 1 se finito senza numero di hit (o se non è partito, es. fermato da controllo soglie)
                int end;\--> 0 finito normalmente, 1 finito per timeout, 2 finito per errori, 3 DAQ crash o non partito

Ad esempio, guardando il run 376 (a cui viene tagliato il 33%)

In [2]:
f = R.TFile.Open("end_of_subruns/run_376")
c2= R.TCanvas("c2")
c1= R.TCanvas("c1")
c = R.TCanvas("c")

Per esempio, i trigger per subrun:

In [3]:
f.tree.Draw("triggers")
c.Draw()

Per vedere come finiscono di solito i subrun:

In [4]:
f.tree.Draw("end")
c.Draw()

Dopo quelli che finisco bene, la popolazione più grossa è quella dei timeout. Vediamoli.

In [5]:
h_ok = R.TH1F( "h_ok", 'triggers in run ok',400 , 0,400) 
h_timeout = R.TH1F( "h_timeout", 'triggers in run timedout',400 , 0,400) 
h_errors = R.TH1F( "h_errors", 'triggers in run with errors',400 , 0,400) 
h_crash = R.TH1F( "h_crash", 'triggers in run with crash',400 , 0,400) 

Non ci vuole nemmeno la legenda perchè la vedete passando il mouse sopra.

In [6]:
for  entryNum  in  range(0,f.tree.GetEntries ()):
    f.tree.GetEntry(entryNum)
    end=getattr(f.tree ,"end")
    triggers=getattr(f.tree ,"triggers")
    if end==0:
        h_ok.Fill(triggers)
    elif end ==1:
        h_timeout.Fill(triggers)

    elif end ==2:
        h_errors.Fill(triggers)

    elif end ==3:
        h_crash.Fill(triggers)

h_ok.Draw()
h_timeout.SetLineColor(2)
h_timeout.Draw("same")

h_errors.SetLineColor(3)
h_errors.Draw("same")

h_crash.SetLineColor(4)
h_crash.Draw("same")
c.Draw()

In [7]:
def total_trig(h):
    integ=0
    for n in range (1, h.GetNbinsX()+1):
        integ+=h.GetBinContent(n)*(h.GetBinCenter(n)-h_ok.GetBarWidth()/2)
    return integ

In [8]:
ok_int=total_trig(h_ok)
timeout_int=total_trig(h_timeout)
errors_int=total_trig(h_errors)
crash_int=total_trig(h_crash)
tot=ok_int+ timeout_int+errors_int+crash_int
print ("Total= {} Triggers in run ok: {}({:.2f}%), triggers in run timeout: {}({:.2f}%), triggers in run errrors: {}({:.2f}%), triggers in run crash {}({:.2f}%):".format(tot, ok_int,ok_int/tot, timeout_int,timeout_int/tot, errors_int,errors_int/tot, crash_int, crash_int/tot))

Total= 82412.0 Triggers in run ok: 72596.0(0.88%), triggers in run timeout: 7016.0(0.09%), triggers in run errrors: 2800.0(0.03%), triggers in run crash 0.0(0.00%):


Torna abbastanza con il numero inviato in aumatico via mail di 83074.76923076923, considerando che sono un migliaio di subrun e ogni tanto una GEMRCO avrà uno o due trigger meno delle altre dandoci il migliaio circa di differneza (passo anche nel codice di acquisizione alla modalità col minimo, è più ragionevole). Non torna per niente col numero mostrato oggi da Riccardo (32847+16676=49523 triggers). <br>
Di questo run vediamo anche quali FEB hanno causato i riavvi per 8bit 10 bit:


In [9]:
c.cd()
f.tree.Draw("error_end_FEB","error_end_FEB>0")
c.Draw()

Almeno da questo punto di vista la presenza delle planari non sembra un grosso handicap

### <hr> Metto tutto in uno script e provo per più run.<hr> <hr> <hr> 

In [10]:
h_ok_s = R.TH1F( "h_ok_s", 'triggers in run ok',400 , 0,400) 
h_timeout_s = R.TH1F( "h_timeout_s", 'triggers in run timedout',400 , 0,400) 
h_errors_s = R.TH1F( "h_errors_s", 'triggers in run with errors',400 , 0,400) 
h_crash_s = R.TH1F( "h_crash_s", 'triggers in run with crash',400 , 0,400) 

In [11]:
def script(run):
    h_ok_s.Reset("ICESM")
    h_timeout_s.Reset("ICESM")
    h_errors_s.Reset("ICESM")
    h_crash_s.Reset("ICESM")
    c1.cd()
    fs = R.TFile.Open("end_of_subruns/run_{}".format(run))
    for  entryNum  in  range(0,fs.tree.GetEntries ()):
        fs.tree.GetEntry(entryNum)
        end=getattr(fs.tree ,"end")
        triggers=getattr(fs.tree ,"triggers")
        if end==0:
            h_ok_s.Fill(triggers)
        elif end ==1:
            h_timeout_s.Fill(triggers)

        elif end ==2:
            h_errors_s.Fill(triggers)

        elif end ==3:
            h_crash_s.Fill(triggers)

    h_ok_s.Draw()
    h_ok_s.SetTitle("triggers per subrun")
    h_timeout_s.SetLineColor(2)
    h_timeout_s.Draw("same")

    h_errors_s.SetLineColor(3)
    h_errors_s.Draw("same")

    h_crash_s.SetLineColor(4)
    h_crash_s.Draw("same")
    ok_int=total_trig(h_ok_s)
    timeout_int=total_trig(h_timeout_s)
    errors_int=total_trig(h_errors_s)
    crash_int=total_trig(h_crash_s)
    tot=ok_int+ timeout_int+errors_int+crash_int
    c2.cd()
    fs.tree.Draw("error_end_FEB","error_end_FEB>0","")
    c2.Draw()
    c1.Draw()

    print ("Total= {} Triggers in run ok: {}({:.2f}%), triggers in run timeout: {}({:.2f}%), triggers in run errrors: {}({:.2f}%), triggers in run crash {}({:.2f}%):".format(tot, ok_int,ok_int/tot, timeout_int,timeout_int/tot, errors_int,errors_int/tot, crash_int, crash_int/tot))

<br> <br> <hr>
#### RUN 377

In [12]:

script(377)

Total= 65414.0 Triggers in run ok: 56676.0(0.87%), triggers in run timeout: 5773.0(0.09%), triggers in run errrors: 2965.0(0.05%), triggers in run crash 0.0(0.00%):


Riccardo stamattina citava 40935 triggers

<hr> <br><hr>Prendiamo un run del quale conosciamo la durata, per verificare quanti trigger ci sono. Per esempio il 395, corto e semplice.<hr>


<br> <br> <hr>
#### RUN 395

In [13]:
script(395)

Total= 6932.0 Triggers in run ok: 5778.0(0.83%), triggers in run timeout: 20.0(0.00%), triggers in run errrors: 1134.0(0.16%), triggers in run crash 0.0(0.00%):


Questo numero torna col numero di subruns (25 subrun * 280 hit a subrun =7000 hit) e col tempo di acquisizione (dalle 20:44 all'1:13, sono 16140 secondi, con un rate di 0.4 hZ vengono 6456 hit). Nel file di Riccardo il totale è di 5425.

<br> <br> <hr>
#### RUN 351

In [14]:
script(351)

Total= 253557.0 Triggers in run ok: 241530.0(0.95%), triggers in run timeout: 1173.0(0.00%), triggers in run errrors: 10854.0(0.04%), triggers in run crash 0.0(0.00%):


Ne vedo quindi 253557 contro i 254001 di Riccardo (ce ne sono di più!). Forse perchè vengono contati anche trigger dove non ci sono tutte le GEMROC (pochi in più fra gli ultimi)

Aggiungo il run 400, appena finito


<br> <br> <hr>
#### RUN 400

In [15]:
script(400)

Total= 37121.0 Triggers in run ok: 34673.0(0.93%), triggers in run timeout: 731.0(0.02%), triggers in run errrors: 1717.0(0.05%), triggers in run crash 0.0(0.00%):


### Conclusioni

La perdita di trigger per problemi di DAQ sembra abbastanza ridotta e inoltre i trigger persi per timeout possono essere reintegrati. <br> Bisogna però capire come mai i numeri complessivi di trigger non coincidono, mi sto dimenticando di qualche taglio?